# This converts relative coordinates (from LLM output) to actual genome coordinates

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import gffutils
from collections import defaultdict
pd.set_option('display.max_colwidth', None)

In [2]:
# input_file = 'inputs/40_nucleotide_deletion_differences_allseqs_names_low_threshold.csv'
# input_file = 'inputs/deletion_effects_threshold_1_30_nucleotides.csv'
# input_file = 'inputs/deletion_effects_effects_human_water_60_nucleotides.csv'
input_file = 'inputs/deletion_effects_effects_human_water_30_nucleotides.csv'
input_dir = 'inputs/'
output_dir = 'outputs/'

## Use GTF to define transcript coordinates 
`regions_positions` contains exon coordinates for each transcript

In [3]:
gtf_header = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
# gtf = pd.read_csv('inputs/Mus_musculus.GRCm39.112.chr.gtf', sep='\t', skiprows=5, names=gtf_header)
gtf = pd.read_csv('inputs/Homo_sapiens.GRCh38.110.chr.gtf', sep='\t', skiprows=5, names=gtf_header)
exons = gtf[gtf['feature']=='exon']
exons['name'] = exons['attribute'].apply(lambda x: x.split('transcript_id \"')[1].split('\";')[0])

exons.head()
exons['start0base'] = exons['start'] - 1
exons['score'] = '0'
exons['chrom'] = 'chr' + exons['seqname'].astype(str)
exons = exons[['chrom', 'start0base', 'end', 'name', 'score', 'strand']]
exons.head()

/tmp/ipykernel_3417782/356886721.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf = pd.read_csv('/tscc/projects/ps-yeolab3/bay001/annotations/GRCh38/ensembl/release_110/Homo_sapiens.GRCh38.110.chr.gtf', sep='\t', skiprows=5, names=gtf_header)
/tmp/ipykernel_3417782/356886721.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exons['name'] = exons['attribute'].apply(lambda x: x.split('transcript_id \"')[1].split('\";')[0])
/tmp/ipykernel_3417782/356886721.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,chrom,start0base,end,name,score,strand
2,chr1,182695,182746,ENST00000624431,0,+
3,chr1,183131,183216,ENST00000624431,0,+
4,chr1,183493,183571,ENST00000624431,0,+
5,chr1,183739,183901,ENST00000624431,0,+
6,chr1,183980,184174,ENST00000624431,0,+


In [4]:
transcripts = gtf[gtf['feature']=='transcript']

def get_transcript_name(row):
    try:
        return row['attribute'].split('transcript_name \"')[1].split('\";')[0]
    except Exception as e:
        return row['attribute'].split('transcript_id \"')[1].split('\";')[0]

transcripts['id'] = transcripts['attribute'].apply(lambda x: x.split('transcript_id \"')[1].split('\";')[0])
transcripts['name'] = transcripts.apply(get_transcript_name, axis=1)
transcripts

/tmp/ipykernel_3417782/4141969072.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts['id'] = transcripts['attribute'].apply(lambda x: x.split('transcript_id \"')[1].split('\";')[0])
/tmp/ipykernel_3417782/4141969072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcripts['name'] = transcripts.apply(get_transcript_name, axis=1)


,seqname,source,feature,start,end,score,strand,frame,attribute,id,name
1,1,havana,transcript,182696,184174,.,+,.,"gene_id ""ENSG00000279928""; gene_version ""2""; transcript_id ""ENST00000624431""; transcript_version ""2""; gene_name ""DDX11L17""; gene_source ""havana""; gene_biotype ""unprocessed_pseudogene""; transcript_name ""DDX11L17-201""; transcript_source ""havana""; transcript_biotype ""unprocessed_pseudogene""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""NA"";",ENST00000624431,DDX11L17-201
8,1,havana,transcript,2581560,2584533,.,+,.,"gene_id ""ENSG00000228037""; gene_version ""1""; transcript_id ""ENST00000424215""; transcript_version ""1""; gene_source ""havana""; gene_biotype ""lncRNA""; transcript_source ""havana""; transcript_biotype ""lncRNA""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""5"";",ENST00000424215,ENST00000424215
13,1,havana,transcript,3069168,3434342,.,+,.,"gene_id ""ENSG00000142611""; gene_version ""17""; transcript_id ""ENST00000511072""; transcript_version ""5""; gene_name ""PRDM16""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""PRDM16-206""; transcript_source ""havana""; transcript_biotype ""protein_coding""; tag ""basic""; transcript_support_level ""5"";",ENST00000511072,PRDM16-206
50,1,havana,transcript,3069183,3186591,.,+,.,"gene_id ""ENSG00000142611""; gene_version ""17""; transcript_id ""ENST00000607632""; transcript_version ""1""; gene_name ""PRDM16""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""PRDM16-210""; transcript_source ""havana""; transcript_biotype ""retained_intron""; transcript_support_level ""2"";",ENST00000607632,PRDM16-210
53,1,havana,transcript,3069197,3435421,.,+,.,"gene_id ""ENSG00000142611""; gene_version ""17""; transcript_id ""ENST00000378391""; transcript_version ""6""; gene_name ""PRDM16""; gene_source ""ensembl_havana""; gene_biotype ""protein_coding""; transcript_name ""PRDM16-203""; transcript_source ""havana""; transcript_biotype ""protein_coding""; tag ""CCDS""; ccds_id ""CCDS44048""; tag ""basic""; transcript_support_level ""1"";",ENST00000378391,PRDM16-203
...,...,...,...,...,...,...,...,...,...,...,...
3421027,MT,insdc,transcript,14149,14673,.,-,.,"gene_id ""ENSG00000198695""; gene_version ""2""; transcript_id ""ENST00000361681""; transcript_version ""2""; gene_name ""MT-ND6""; gene_source ""insdc""; gene_biotype ""protein_coding""; transcript_name ""MT-ND6-201""; transcript_source ""insdc""; transcript_biotype ""protein_coding""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""NA"";",ENST00000361681,MT-ND6-201
3421033,MT,insdc,transcript,14674,14742,.,-,.,"gene_id ""ENSG00000210194""; gene_version ""1""; transcript_id ""ENST00000387459""; transcript_version ""1""; gene_name ""MT-TE""; gene_source ""insdc""; gene_biotype ""Mt_tRNA""; transcript_name ""MT-TE-201""; transcript_source ""insdc""; transcript_biotype ""Mt_tRNA""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""NA"";",ENST00000387459,MT-TE-201
3421036,MT,insdc,transcript,14747,15887,.,+,.,"gene_id ""ENSG00000198727""; gene_version ""2""; transcript_id ""ENST00000361789""; transcript_version ""2""; gene_name ""MT-CYB""; gene_source ""insdc""; gene_biotype ""protein_coding""; transcript_name ""MT-CYB-201""; transcript_source ""insdc""; transcript_biotype ""protein_coding""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""NA"";",ENST00000361789,MT-CYB-201
3421041,MT,insdc,transcript,15888,15953,.,+,.,"gene_id ""ENSG00000210195""; gene_version ""2""; transcript_id ""ENST00000387460""; transcript_version ""2""; gene_name ""MT-TT""; gene_source ""insdc""; gene_biotype ""Mt_tRNA""; transcript_name ""MT-TT-201""; transcript_source ""insdc""; transcript_biotype ""Mt_tRNA""; tag ""basic""; tag ""Ensembl_canonical""; transcript_support_level ""NA"";",ENST00000387460,MT-TT-201


In [5]:
tx2name = transcripts[['id', 'name']].set_index('id')['name'].to_dict()

In [7]:
exons

,chrom,start0base,end,name,score,strand
2,chr1,182695,182746,ENST00000624431,0,+
3,chr1,183131,183216,ENST00000624431,0,+
4,chr1,183493,183571,ENST00000624431,0,+
5,chr1,183739,183901,ENST00000624431,0,+
6,chr1,183980,184174,ENST00000624431,0,+
...,...,...,...,...,...,...
3421028,chrMT,14148,14673,ENST00000361681,0,-
3421034,chrMT,14673,14742,ENST00000387459,0,-
3421037,chrMT,14746,15887,ENST00000361789,0,+
3421042,chrMT,15887,15953,ENST00000387460,0,+


In [8]:
regions = exons

In [9]:
regions['transcript_id'] = regions['name'].apply(lambda x: x.split('.')[0])  # in case we use gencode
regions.head()

,chrom,start0base,end,name,score,strand,transcript_id
2,chr1,182695,182746,ENST00000624431,0,+,ENST00000624431
3,chr1,183131,183216,ENST00000624431,0,+,ENST00000624431
4,chr1,183493,183571,ENST00000624431,0,+,ENST00000624431
5,chr1,183739,183901,ENST00000624431,0,+,ENST00000624431
6,chr1,183980,184174,ENST00000624431,0,+,ENST00000624431


In [10]:
regions.sort_values(by=['transcript_id','start0base','end'], inplace=True)  # critical for appending in order.
regions_positions = defaultdict(list)
for row in regions.iterrows():
    regions_positions[row[1]['transcript_id']].append({'chrom': row[1]['chrom'], 'start': row[1]['start0base'], 'end': row[1]['end'], 'strand': row[1]['strand']})

In [11]:
# regions_positions['ENSMUST00000134384']
regions_positions['ENST00000624431']

[{'chrom': 'chr1', 'start': 182695, 'end': 182746, 'strand': '+'},
 {'chrom': 'chr1', 'start': 183131, 'end': 183216, 'strand': '+'},
 {'chrom': 'chr1', 'start': 183493, 'end': 183571, 'strand': '+'},
 {'chrom': 'chr1', 'start': 183739, 'end': 183901, 'strand': '+'},
 {'chrom': 'chr1', 'start': 183980, 'end': 184174, 'strand': '+'}]

In [12]:
missing_name = set()
def get_transcript_name(row, tx2name=tx2name):
    try:
        return tx2name[row['transcript_id']]
    except KeyError:
        missing_name.add(row['transcript_id'])
        return row['transcript_id']
        
deletions = pd.read_csv(input_file)
if 'external_gene_name' not in deletions.columns:
    deletions['external_gene_name'] = deletions.apply(get_transcript_name, axis=1)

print(f"Number of transcripts missing gene names: {len(missing_name)}")
deletions.head()

Number of transcripts missing gene names: 4


,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,external_gene_name
0,ENST00000000233,0,30,0.000000,0.044434,ARF5-201
1,ENST00000000233,15,45,0.000000,0.044434,ARF5-201
2,ENST00000000233,30,60,0.000000,0.044434,ARF5-201
3,ENST00000000233,45,75,0.000000,0.044434,ARF5-201
4,ENST00000000233,60,90,-0.001953,0.042480,ARF5-201


In [13]:
missing_name  # due to version mismatches in my GTF and Karen's. Should not be many but drop those that don't exist.

{'ENST00000612465', 'ENST00000612640', 'ENST00000613204', 'ENST00000614336'}

In [14]:
from itertools import groupby

def collapse_ranges_groupby(data):
    """
    Collapse sequences of numbers in a list into ranges using groupby.
    
    Args:
      data: A list of integers.
    
    Returns:
      A list of ranges represented as tuples (start, end).
    """
    for _, group in groupby(enumerate(data), lambda pair: pair[1] - pair[0]):
        group = list(group)
        yield f"{group[0][1]}-{group[-1][1]+1}"
      
def get_gene_coords_from_tx_id(tx_id, relstart, relend, pos=regions_positions):

    intervals = pos[tx_id]
    if len(intervals) == 0: return ""  # utr not found
    chrom = intervals[0]['chrom']
    strand = list(set([s['strand'] for s in intervals]))
    try:
        assert len(strand) == 1
        strand = strand[0]
    except AssertionError:  # One transcript can't be associated with multiple strands?
        # print(f"{tx_id} exons are associated with more than one strand: {intervals}")
        return ""
    # print(intervals)
    utr_coords = [i for u in intervals for i in range(u['start'], u['end'])]
    # print(utr_coords)
    if len(utr_coords) < relend-relstart: return ""
    if strand == '+':
        true_coords = utr_coords[relstart:relend] 
        # print(true_coords)
    elif strand == '-':
        if relstart == 0:  # -0 breaks lists
            true_coords = utr_coords[-relend:]
        else:
            true_coords = utr_coords[-relend:-relstart]
    return ','.join([f"{chrom}:{pos}:{strand}" for pos in list(collapse_ranges_groupby(true_coords))])

get_gene_coords_from_tx_id(tx_id='ENST00000624431', relstart=60, relend=100, pos=regions_positions)

'chr1:183140-183180:+'

### Some unit tests to ensure code is correct

In [15]:
test_positions = {}
test_positions['test_pos_1'] = [
    {'chrom': 'chr1', 'start': 1000, 'end': 1080, 'strand': '+'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2100, 'strand': '+'}
]
test_positions['test_pos_2'] = [
    {'chrom': 'chr1', 'start': 1000, 'end': 1020, 'strand': '+'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2010, 'strand': '+'},
    {'chrom': 'chr1', 'start': 3000, 'end': 3050, 'strand': '+'},
]
test_positions['test_pos_3'] = [
    {'chrom': 'chr1', 'start': 1000, 'end': 1020, 'strand': '+'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2010, 'strand': '+'},
    {'chrom': 'chr1', 'start': 3000, 'end': 3050, 'strand': '+'},
]
test_positions['test_pos_4'] = [
    {'chrom': 'chr1', 'start': 1000, 'end': 1080, 'strand': '-'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2100, 'strand': '-'}
]
test_positions['test_pos_5'] = [
    {'chrom': 'chr1', 'start': 0, 'end': 1020, 'strand': '-'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2010, 'strand': '-'},
    {'chrom': 'chr1', 'start': 3000, 'end': 3050, 'strand': '-'},
]
test_positions['test_pos_6'] = [
    {'chrom': 'chr1', 'start': 1000, 'end': 1020, 'strand': '-'},
    {'chrom': 'chr1', 'start': 2000, 'end': 2010, 'strand': '-'},
    {'chrom': 'chr1', 'start': 3000, 'end': 3050, 'strand': '-'},
]
print(get_gene_coords_from_tx_id(tx_id='test_pos_1', relstart=40, relend=81, pos=test_positions)) # chr1:1040-1080:+,chr1:2000-2001:+
print(get_gene_coords_from_tx_id(tx_id='test_pos_2', relstart=40, relend=81, pos=test_positions)) # chr1:3010-3050:+
print(get_gene_coords_from_tx_id(tx_id='test_pos_3', relstart=25, relend=44, pos=test_positions)) # chr1:2005-2010:+,chr1:3000-3014:+

print(get_gene_coords_from_tx_id(tx_id='test_pos_4', relstart=40, relend=81, pos=test_positions)) # chr1:2019-2060:-
print(get_gene_coords_from_tx_id(tx_id='test_pos_5', relstart=40, relend=81, pos=test_positions)) # chr1:999-1020:-,chr1:2000-2010:-,chr1:3000-3010:-
print(get_gene_coords_from_tx_id(tx_id='test_pos_6', relstart=25, relend=44, pos=test_positions)) # chr1:3006-3025:-

chr1:1040-1080:+,chr1:2000-2001:+
chr1:3010-3050:+
chr1:2005-2010:+,chr1:3000-3014:+
chr1:2019-2060:-
chr1:999-1020:-,chr1:2000-2010:-,chr1:3000-3010:-
chr1:3006-3025:-


## Iterate over each row in deletion and return true coordinates from relative ones

In [ ]:
genomic_coords = {}
for row in deletions.iterrows():
    genomic_coords[row[0]] = get_gene_coords_from_tx_id(tx_id=row[1]['transcript_id'], relstart=row[1]['deletion_index_start'], relend=row[1]['deletion_index_end'])

In [17]:
genomic_coords = pd.DataFrame(genomic_coords, index=['coords']).T
genomic_coords.head()

,coords
0,chr7:127588410-127588440:+
1,chr7:127588425-127588455:+
2,chr7:127588440-127588470:+
3,chr7:127588455-127588485:+
4,chr7:127588470-127588500:+


## Merge true coordinates with deletions table

In [18]:
merged = pd.merge(deletions, genomic_coords, how='outer', left_index=True, right_index=True)
merged

,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,external_gene_name,coords
0,ENST00000000233,0,30,0.000000,0.044434,ARF5-201,chr7:127588410-127588440:+
1,ENST00000000233,15,45,0.000000,0.044434,ARF5-201,chr7:127588425-127588455:+
2,ENST00000000233,30,60,0.000000,0.044434,ARF5-201,chr7:127588440-127588470:+
3,ENST00000000233,45,75,0.000000,0.044434,ARF5-201,chr7:127588455-127588485:+
4,ENST00000000233,60,90,-0.001953,0.042480,ARF5-201,chr7:127588470-127588500:+
...,...,...,...,...,...,...,...
6969881,ENST00000707136,2835,2865,0.000000,0.018433,PRRT3-205,chr3:9939588-9939618:-
6969882,ENST00000707136,2850,2880,0.000000,0.018433,PRRT3-205,chr3:9939573-9939603:-
6969883,ENST00000707136,2865,2895,0.000000,0.018433,PRRT3-205,chr3:9939558-9939588:-
6969884,ENST00000707136,2880,2910,0.000000,0.018433,PRRT3-205,chr3:9939543-9939573:-


In [19]:
# Double check hotspots that span more than one exon
merged[merged['coords'].str.contains(',')].head(50)

,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,external_gene_name,coords
9,ENST00000000233,135,165,0.000977,0.045410,ARF5-201,"chr7:127588545-127588565:+,chr7:127589082-127589092:+"
10,ENST00000000233,150,180,0.014893,0.059326,ARF5-201,"chr7:127588560-127588565:+,chr7:127589082-127589107:+"
14,ENST00000000233,210,240,0.000000,0.044434,ARF5-201,"chr7:127589137-127589163:+,chr7:127589484-127589488:+"
15,ENST00000000233,225,255,0.000000,0.044434,ARF5-201,"chr7:127589152-127589163:+,chr7:127589484-127589503:+"
22,ENST00000000233,330,360,0.000000,0.044434,ARF5-201,"chr7:127589578-127589594:+,chr7:127590065-127590079:+"
23,ENST00000000233,345,375,0.000000,0.044434,ARF5-201,"chr7:127589593-127589594:+,chr7:127590065-127590094:+"
26,ENST00000000233,390,420,0.001709,0.046143,ARF5-201,"chr7:127590109-127590137:+,chr7:127590962-127590964:+"
27,ENST00000000233,405,435,0.001953,0.046387,ARF5-201,"chr7:127590124-127590137:+,chr7:127590962-127590979:+"
35,ENST00000000233,525,555,0.001221,0.045654,ARF5-201,"chr7:127591069-127591088:+,chr7:127591212-127591223:+"
36,ENST00000000233,540,570,0.001221,0.045654,ARF5-201,"chr7:127591084-127591088:+,chr7:127591212-127591238:+"


In [20]:
# merged.to_csv(os.path.join(output_dir, '40_nucleotide_deletion_differences_allseqs_names_low_threshold.coords.tsv'), sep='\t', index=False, header=True)
# merged.to_csv(os.path.join(output_dir, 'deletion_effects_threshold_1_30_nucleotides.coords.tsv'), sep='\t', index=False, header=True)
merged.to_csv(os.path.join(output_dir, os.path.basename(input_file).replace('.csv', '.coords.tsv')), sep='\t', index=False, header=True)

## Check once more that regions spanning multiple exons are correct
- Here we check regions that span at least three exons

In [21]:
merged['num_exons'] = merged['coords'].apply(lambda x: x.count(',')+1)
merged[merged['num_exons'] > 2]

,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,external_gene_name,coords,num_exons
6271,ENST00000011619,1230,1260,0.000000,0.014038,RANBP9-201,"chr6:13644726-13644729:-,chr6:13652658-13652681:-,chr6:13657108-13657112:-",3
8725,ENST00000013125,2010,2040,0.000793,0.016235,MAP4K5-201,"chr14:50437995-50438008:-,chr14:50438091-50438094:-,chr14:50440012-50440026:-",3
19034,ENST00000064778,435,465,-0.000488,0.014038,FAM168A-201,"chr11:73419997-73419999:-,chr11:73425021-73425048:-,chr11:73430689-73430690:-",3
43148,ENST00000195649,1170,1200,0.000000,0.013428,SNAP91-201,"chr6:83610675-83610677:-,chr6:83614856-83614862:-,chr6:83616968-83616990:-",3
43149,ENST00000195649,1185,1215,0.000000,0.013428,SNAP91-201,"chr6:83610660-83610677:-,chr6:83614856-83614862:-,chr6:83616968-83616975:-",3
...,...,...,...,...,...,...,...,...
6964154,ENST00000706889,4710,4740,0.000244,0.016357,MADD-227,"chr11:47324565-47324577:+,chr11:47326543-47326552:+,chr11:47326737-47326746:+",3
6965958,ENST00000706935,1500,1530,0.000305,0.009705,INPP4A-210,"chr2:98546687-98546694:+,chr2:98548951-98548969:+,chr2:98552785-98552790:+",3
6967922,ENST00000707119,1695,1725,-0.000244,0.013611,ABLIM1-215,"chr10:114439245-114439250:-,chr10:114440081-114440089:-,chr10:114441016-114441033:-",3
6967923,ENST00000707119,1710,1740,-0.000427,0.013428,ABLIM1-215,"chr10:114439230-114439250:-,chr10:114440081-114440089:-,chr10:114441016-114441018:-",3
